In [34]:
import spacy
import textacy
from textacy import lexicon_methods
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import csv
import os
import sklearn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import wordnet as wn
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords 
import math
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.utils import to_categorical
import numpy as np
from numpy import array 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()
from spacy.parts_of_speech import ADJ, ADV, NOUN, VERB
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D
from keras.models import Sequential
from keras.layers import Input, Flatten
from keras.layers import Dense, Activation
from keras.layers import MaxPooling1D
from os import path
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from spacy.lang.en import English
from ahocorasick import Automaton
from spacy.util import minibatch, compounding

In [ ]:
'''train a model  to categorize the reviews based on positive and negative sentiments. So output of svm will be 1  and 1
0-negative
Take the negative sentiment reviews and apply ner
These are the reviews that have issues
Once the entities have been recognised, the review can be routed to the corresponding dept
We can show it in a table -review, corresponding department
And another table that can have review, corresponding response.
Ex:  Review: few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead. 
     Response :What was the brand of the top?
'''

In [ ]:
#Categories:
#1. Clothing and Accessories
#2. Cell_Phones_Accessories.txt
#3. Home_Kitchen.txt

In [13]:
with open('/Users/pragnyasuresh/Downloads/Clothing_Accessories.txt', 'r') as f:
    with open('/Users/pragnyasuresh/Desktop/nlp_project/reviews.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        header=["department","sentiment","review"]
        writer.writerow(header)
        ctr=0
        for line in f:
            if(line.find("review/score:")!=-1):
                    
                    rev=["Clothing_Accessories",line[13:]]
            
            
            
            if(line.find("review/text:")!=-1 and ctr<4000):
                
                review=str(line[13:]).lower()
                #print(review)
                rev.append(review)
                
                #print(rev)
                
                writer = csv.writer(writeFile)
                writer.writerow(rev)
                ctr+=1
            
            
    print(ctr)

4000


In [14]:
with open('/Users/pragnyasuresh/Downloads/Cell_Phones_Accessories.txt', 'r') as f:
    with open('/Users/pragnyasuresh/Desktop/nlp_project/reviews.csv', 'a') as writeFile:

       

        ctr=0
        for line in f:
            
            if(line.find("review/score:")!=-1):
                    
                    rev=["Cell_Phones_Accessories",line[13:]]
            
            
            
            if(line.find("review/text:")!=-1 and ctr<4000):
                
            
                review=str(line[13:]).lower()
                #print(review)
                rev.append(review)
                
                #print(rev)
                
                writer = csv.writer(writeFile)
                writer.writerow(rev)
                ctr+=1
            
            
    print(ctr)

4000


In [15]:
with open('/Users/pragnyasuresh/Downloads/Home_Kitchen.txt', 'r') as f:
    with open('/Users/pragnyasuresh/Desktop/nlp_project/reviews.csv', 'a') as writeFile:

       
        ctr=0
        for line in f:
            
            if(line.find("review/score:")!=-1):
                    
                    rev=["Home_Kitchen",line[13:]]
            
            
            
            if(line.find("review/text:")!=-1 and ctr<4000):
                
                review=str(line[13:]).lower()
                #print(review)
                rev.append(review)
                
                #print(rev)
                
                writer = csv.writer(writeFile)
                writer.writerow(rev)
                ctr+=1
            
            
    print(ctr)

4000


In [18]:
df=pd.read_csv('/Users/pragnyasuresh/Desktop/nlp_project/reviews.csv', delimiter = ',')
print(len(df))
print(df['sentiment'].value_counts())
#Drop all the reviews with rating 3

df = df[df.sentiment != 3]
print(len(df))
df['sentiment']=df['sentiment'].replace([1, 2], 0)
df['sentiment']=df['sentiment'].replace([4, 5], 1)
print(df['sentiment'].value_counts())


12000
5.0    6064
4.0    2349
1.0    1530
3.0    1120
2.0     937
Name: sentiment, dtype: int64
10880
1.0    8413
0.0    2467
Name: sentiment, dtype: int64


In [19]:
#balance
df1=df.loc[df['sentiment'] == 1]
df2=df.loc[df['sentiment'] == 0]

df1=df1[:3773]
print(len(df1),len(df2))

df=df1.append(df2, ignore_index=True)
print(df['department'].value_counts())

3773 2467
Clothing_Accessories       3650
Cell_Phones_Accessories    2021
Home_Kitchen                569
Name: department, dtype: int64


In [20]:
#Creating sparse vector using TFID vectorizer
vectorizer = TfidfVectorizer()
sparse_vector = vectorizer.fit_transform(df["review"])
n_features=sparse_vector.shape[1]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(sparse_vector, df["sentiment"], test_size=0.4, random_state=42)
l1=list(df["department"])
l2=list(df["review"])
l=[]
for i in range(len(l1)):
    l.append([l1[i],l2[i]])
#print(len(l))
#need to map the reviews and department to the review. To do this, the line below is used. It will split the same way
X_train2, X_test2, y_train2, y_test2 = train_test_split(l, df["sentiment"], test_size=0.4, random_state=42)
clf = SVC(kernel='linear') 
# fitting x samples and y classes 
clf.fit(X_train, y_train) 
#Predicting and evaluatng accuracy

y_pred_list=clf.predict(X_test)
ans=array(y_pred_list)
acc = sklearn.metrics.accuracy_score(y_test, y_pred_list)
print("Accuracy using sparse vector representation of each review =",acc*100)
#Since we needed to manually annotate the corpus for training the ner, we could not use a huge corpus
#NN needs huge amount of data to provide good accuracy.
#SVM classification gave much better accuracy whereas ANN gave about 72% so we decided to use SVM to classify the reviews

Accuracy using sparse vector representation of each review = 92.90865384615384


In [22]:
#we now take all reviews with a negative sentiment
y_pred_list=list(y_pred_list)
df_pred = pd.DataFrame({'sent':y_pred_list})

print (df_pred["sent"].value_counts())
X_test2=list(X_test2)
negative_reviews=[]
j=0
with open('/Users/pragnyasuresh/Desktop/nlp_project/negative_reviews.csv', 'w') as writeFile:
    header=["department","review"]
    writer = csv.writer(writeFile)
    writer.writerow(header)
    for i in y_pred_list:
    
        if(i==0.0):
            #print(i,j)
            negative_reviews=[X_test2[j][0]]
            negative_reviews.append(X_test2[j][1])
            #print(negative_reviews)
            writer = csv.writer(writeFile)
            writer.writerow(negative_reviews)
        j+=1
df_neg = pd.read_csv('/Users/pragnyasuresh/Desktop/nlp_project/negative_reviews.csv') 
df_neg["department"].value_counts()

1.0    1538
0.0     958
Name: sent, dtype: int64


Cell_Phones_Accessories    480
Clothing_Accessories       255
Home_Kitchen               223
Name: department, dtype: int64

In [47]:
#perform ner on negative_reviews


In [23]:
df_neg['department']=df_neg['department'].replace("Clothing_Accessories", 1)
df_neg['department']=df_neg['department'].replace("Cell_Phones_Accessories", 2)
df_neg['department']=df_neg['department'].replace("Home_Kitchen", 3)


vectorizer = TfidfVectorizer()
sparse_vector_neg = vectorizer.fit_transform(df_neg["review"])
X_train, X_test, y_train, y_test = train_test_split(sparse_vector_neg, df_neg["department"], test_size=0.4, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(df_neg["review"], df_neg["department"], test_size=0.4, random_state=42)

clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(X_train,y_train) 

y_pred_list=clf.predict(X_test)
ans=array(y_pred_list)
acc = sklearn.metrics.accuracy_score(y_test, y_pred_list)
print("Accuracy using sparse vector representation of each review =",acc*100)

Accuracy using sparse vector representation of each review = 83.33333333333334


In [29]:
df_neg_dept = pd.DataFrame({'dept':y_pred_list,'rev':X_test2})
df_neg_dept.to_csv("/Users/pragnyasuresh/Desktop/nlp_project/classified_reviews.csv", encoding='utf-8')

In [35]:
A=Automaton()
A.add_word("Roadster",('Brand','Roadster'))
A.add_word("top",('Product','top'))
A.add_word("nylon",('INFO','nylon'))
A.add_word("cotton",('INFO','cotton'))
A.add_word("small",('INFO','small'))
A.add_word("large",('INFO','large'))
 
A.make_automaton()
e=["I recently bought a top from the brand Roadster","The size of the top was small", "A few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead.","I did not like the top that I got","The small size itself was large"]
for x in e:
    for item in A.iter(x):
        print(item, x)

(22, ('Product', 'top')) I recently bought a top from the brand Roadster
(46, ('Brand', 'Roadster')) I recently bought a top from the brand Roadster
(18, ('Product', 'top')) The size of the top was small
(28, ('INFO', 'small')) The size of the top was small
(36, ('INFO', 'cotton')) A few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead.
(40, ('Product', 'top')) A few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead.
(72, ('INFO', 'nylon')) A few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead.
(76, ('Product', 'top')) A few weeks ago I ordered some cotton tops. They sent me some cheap nylon tops instead.
(21, ('Product', 'top')) I did not like the top that I got
(8, ('INFO', 'small')) The small size itself was large
(30, ('INFO', 'large')) The small size itself was large
